In [ ]:
import Pkg; Pkg.activate("../scripts/")
Pkg.instantiate()

In [ ]:
using DataFrames, JLD2, IntervalArithmetic

In [ ]:
JLD2.@load "/home/isaia/Code/PlateauExperiment.jl/scripts/results_NIO.jld2" df

In [ ]:
using DataFrames, IntervalArithmetic

function detect_Lyapunov_crossings(df::DataFrame)
    result = DataFrame(alpha1=Float64[], sigma1=Float64[], lambda1=Interval[],
                       alpha2=Float64[], sigma2=Float64[], lambda2=Interval[])

    for α in unique(df.alpha)
        rows = filter(row -> row.alpha == α, df)
        sorted_rows = sort(rows, :sigma)

        for i in 1:(nrow(sorted_rows)-1)
            λ1 = sorted_rows.lambda[i]
            λ2 = sorted_rows.lambda[i+1]

            if sup(λ1) > 0 && inf(λ2) < 0
                push!(result, (
                    alpha1 = α,
                    sigma1 = sorted_rows.sigma[i],
                    lambda1 = λ1,
                    alpha2 = α,
                    sigma2 = sorted_rows.sigma[i+1],
                    lambda2 = λ2
                ))
            end
        end
    end
    return result
end

In [ ]:
result = detect_Lyapunov_crossings(df)

In [ ]:
Pkg.add("Plots")
using Plots

In [ ]:
function plot_lyapunov_transitions(crossings_df::DataFrame)
    λ1_mids = mid.(crossings_df.lambda1)
    α_mids = mid.(crossings_df.alpha1)
    σ_mids = mid.(crossings_df.sigma1)

    scatter(
        α_mids,
        σ_mids;
        markercolor = :blue,
        xlabel = "α",
        label = "σ_low",
        title = "Noise-Induced Order Transitions",
        markersize = 1,
        markerstrokewidth = 0,
        legend = false,
    )
    α_mids = mid.(crossings_df.alpha2)
    σ_mids = mid.(crossings_df.sigma2)
    scatter!(
        α_mids,
        σ_mids;
        markercolor = :red,
        xlabel = "α",
        label = "σ_up",
        markersize = 1,
        markerstrokewidth = 0,
        legend = false,
    )
end

In [ ]:
plot_lyapunov_transitions(result)

In [ ]:
savefig("lyapunov_transitions.png")

In [ ]:
function plot_lambda_signs(df::DataFrame)
    α_vals = mid.(df.alpha)
    σ_vals = mid.(df.sigma)
    λ_vals = mid.(df.lambda)

    # Build color vector: red if λ > 0, blue if λ < 0
    colors = map(λ -> λ > 0 ? :red : :blue, λ_vals)

    scatter(
        σ_vals,
        α_vals;
        markercolor = colors,
        markersize = 1,
        markerstrokewidth = 0,
        xlabel = "α",
        ylabel = "σ",
        legend = false,
        title = "Lyapunov Sign: Red = Positive, Blue = Negative"
    )
end

In [ ]:
plot_lambda_signs(df)

In [ ]:
Pkg.rm("Plots")